Drew Lickman\
CSCI 4820-001\
Project #3\
Due: 10/9/24

AI Usage Disclaimer:


# Lexicon-Based Sentiment Analysis using Custom Logistic Regression

## Assignment Requirements:

### Input
---

- Positive words
- Negative words
- IMDb reviews

### Processing
---

- There are two classifiers
	- Custom Logistic Regression
	- sklearn LogisticRegression
- Implement a Python class (CustomLogisticRegression)
	- \__init\__(self, learning_rate, num_iters) method
		- self.learning_rate
		- self.num_iters
		- self.weights = None
		- self.bias = None
	- sigmoid(z)
		- return result
	- fit(X, y)
		- Sets weights to correct shape and initializes them to 0
		- Applies batch gradient descent to the entire dataset in a loop for num_iters
	- predict(X)
		- z = w dot x + b
		- return sigmoid(z) 

### Output
---

- For each trial and for each classifier
	- Print the sklearn confusion_matrix and classification_Report
- Output the average of the confusion matrices across trials for each classifier

## Python Code

Markdown above each cell used to explain each block of code

1. Load and preprocess IMDb reviews

In [39]:
import numpy as np

sentimentWords = {}
posWords = {}
negWords = {}
with open("positive-words.txt", encoding="utf-8") as positivewords:
	lines = positivewords.readlines()
	for line in lines:
		if line[0] != ";" and line.strip() != '': 
			posWords[line.rstrip('\n')] = 1
with open("negative-words.txt", encoding="utf-8") as negativewords:
	lines = negativewords.readlines()
	for line in lines:
		if line[0] != ";" and line.strip() != '':
			negWords[line.rstrip('\n')] = 1
sentimentWords = {**posWords, **negWords} # Combine positive words and negative words into one dictionary
#print(sentimentWords)

# Add each line of the IMDb reviews to the reviews array
reviews = []
trueValues = []
with open("imdb_reviews.txt", encoding="utf-8") as imdbreviews:
	lines = imdbreviews.readlines()
	for line in lines:
		splitLine = line.rstrip().rsplit(' ', 1)
		reviews.append(splitLine[0]) # removes true sentiment label from data

		sentiment = splitLine[1].strip()[-8:] # the last 8 characters are either positive or negative
		if sentiment == "positive":
			trueValues.append(1)
		elif sentiment == "negative":
			trueValues.append(0)
		else:
			print("Error: sentiment analysis not found at end of line!")
print(reviews[0])
print(trueValues)

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn 
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 

2. Create Features(X) table and Labels(y) array

In [40]:
X = np.zeros((len(reviews), len(sentimentWords)), dtype=bool) 	# Features
y = np.zeros(len(reviews), dtype=int) 							# Labels
y = np.array(trueValues, dtype=int)
# Count how many positive / negative words show up in each review
posCount = 0
negCount = 0

# If a sentiment word is in the review, mark it as True in the X feature table
for review in range(len(reviews)):
	for word in sentimentWords:
		if word in reviews[review]:
			X[review, sentimentWords[word]] = True
	# for posWord in posWords:
	# 	if posWord in reviews[review]: #make two loops, one for pos/neg
	# 		X[review, posWords[posWord]] = True
	# 		posCount += 1 	
	# for negWord in negWords:
	# 	if negWord in reviews[review]:
	# 		X[review, negWords[negWord]] = True #issue
	# 		negCount += 1

	#print(posCount, negCount)

	# if posCount >= negCount:
	# 	y[review] = 1
	# else:
	# 	y[review] = 0
	# posCount = 0
	# negCount = 0


print(X.shape)
print(y.shape)
# for review in range(10):
# 	print(y[review], end=" ")
# 	print(X[review])
	#if y[review]:
		#print(f"Review {review} is positive!")

# Compare sentiment count compared to true value
# count = 0
# for i in range(len(y)):
# 	#print(y[i], end=" ")
# 	#print(trueValues[i])
# 	if y[i] == trueValues[i]: # Count how many pos/negCount labels match the trueValue in each review line
# 		count+=1
# print(f"{count} out of {len(y)} are matching. {count/len(y)*100}%")


(25000, 6786)
(25000,)


3. Define Custom Logistic Regression class

In [41]:
class CustomLogisticRegression():
	# Constructor
	def __init__(self, learning_rate, num_iters): 
		self.learning_rate = learning_rate
		self.num_iters = num_iters
		self.weights = None
		self.bias = None

	# Train the model using gradient descent
	# X is training features, y is labels
	def fit(self, X, y):
		# Sets the weights to the correct shape and initializes them to 0
		features = X.shape[1]
		self.weights = np.zeros(features) # weight for each feature
		self.bias = 0

		# Apply batch gradient descent on entire dataset
		# Gradient descent
		# This for loop was written by Claude 3.5 Sonnet and modified by myself
		for _ in range(self.num_iters):
			predictions = self.sigmoid(self.linearTransform(X)) # Calculate array of sigmoidal probabilities
			error = predictions - y # Calculate the difference between predicted and actual labels

			# Compute gradient for weights
			dw = (1 / len(y)) * np.dot(X.T, error) # X.T is transposed 
			# Compute gradient for bias
			db = (1 / len(y)) * np.sum(error) # Average of all errors

			# Update weights and biases
			self.weights -= self.learning_rate * dw
			self.bias -= self.learning_rate * db

	# Inputs either scalar or array and outputs sigmoid function of the scalar or array
	def sigmoid(self, z):
		output = 1 / (1 + np.exp(-z)) # np.exp does e^(-z) for all samples in the reviews array
		return output

	# Calculate probability of a sample being a class (positive or negative)
	def predict(self, X):
		z = self.linearTransform(X)
		prob = self.sigmoid(z)
		prob = int(prob >= 0.5) # Convert to binary output
		return prob
	
	# Function for X dot W + b
	def linearTransform(self, X):
		z = np.dot(X, self.weights) + self.bias
		return z

scikit-learn documentation
	
	- https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict

	- https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.confusion_matrix.html
	
	- https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.classification_report.html

In [42]:
from sklearn import linear_model as lm
from sklearn import model_selection as ms
from sklearn import metrics

# Initialize variables to store average confusion matrices
avgConfusionMatrix_skllr = np.zeros((2, 2))
avgConfusionMatrix_mylr = np.zeros((2, 2))

trialCount = 5
iterationCount = 500
for trial in range(trialCount):
	# Shuffle input data
	# Split into 80% 20% split of training and test sets
	# Line from Claude 3.5 Sonnet
	X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=trial)

	skllr = lm.LogisticRegression(solver='sag', C=0.001, max_iter=iterationCount)
	skllr.fit(X_train, y_train) # Only use the 80% of the data marked for training
	skllrPredictions = skllr.predict(X_test) # Use the remaining 20% of the data marked for testing
	
	mylr = CustomLogisticRegression(learning_rate=0.1, num_iters=iterationCount)
	mylr.fit(X_train, y_train) # Only use the 80% of the data marked for training
	mylrPredictions = np.array([mylr.predict(x) for x in X_test]) # Use the remaining 20% of the data marked for testing
	
	# for i in skllrPredictions:
	# 	print(skllrPredictions[i], end=" ")
	# 	print(mylrPredictions[i])
	
	# Evaluate sklearn model
	print(f"Trial {trial + 1} - Sklearn LogisticRegression:")
	print(metrics.confusion_matrix(y_test, skllrPredictions))
	print(metrics.classification_report(y_test, skllrPredictions, target_names=["Positive", "Negative"]))

	# Evaluate custom model
	print(f"Trial {trial + 1} - Custom LogisticRegression:")
	print(metrics.confusion_matrix(y_test, mylrPredictions))
	print(metrics.classification_report(y_test, mylrPredictions, target_names=["Positive", "Negative"]))

	# Update average confusion matrices
	avgConfusionMatrix_skllr += metrics.confusion_matrix(y_test, skllrPredictions)
	avgConfusionMatrix_mylr += metrics.confusion_matrix(y_test, mylrPredictions)
# Calculate and print average confusion matrices
avgConfusionMatrix_skllr /= 5
avgConfusionMatrix_mylr /= 5

# After all trials are completed, print average of the trials
print("Average Confusion Matrix - Sklearn LogisticRegression:")
print(avgConfusionMatrix_skllr)

print("Average Confusion Matrix - Custom LogisticRegression:")
print(avgConfusionMatrix_mylr)

Trial 1 - Sklearn LogisticRegression:
[[   0 2518]
 [   0 2482]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2518
    Negative       0.50      1.00      0.66      2482

    accuracy                           0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.33      5000

Trial 1 - Custom LogisticRegression:
[[   0 2518]
 [   0 2482]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2518
    Negative       0.50      1.00      0.66      2482

    accuracy                           0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.33      5000



C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

Trial 2 - Sklearn LogisticRegression:
[[   0 2542]
 [   0 2458]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2542
    Negative       0.49      1.00      0.66      2458

    accuracy                           0.49      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.24      0.49      0.32      5000

Trial 2 - Custom LogisticRegression:
[[   0 2542]
 [   0 2458]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2542
    Negative       0.49      1.00      0.66      2458

    accuracy                           0.49      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.24      0.49      0.32      5000



C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

Trial 3 - Sklearn LogisticRegression:
[[   0 2516]
 [   0 2484]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2516
    Negative       0.50      1.00      0.66      2484

    accuracy                           0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.33      5000

Trial 3 - Custom LogisticRegression:
[[   0 2516]
 [   0 2484]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2516
    Negative       0.50      1.00      0.66      2484

    accuracy                           0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.33      5000



C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

Trial 4 - Sklearn LogisticRegression:
[[2457    0]
 [2543    0]]
              precision    recall  f1-score   support

    Positive       0.49      1.00      0.66      2457
    Negative       0.00      0.00      0.00      2543

    accuracy                           0.49      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.24      0.49      0.32      5000

Trial 4 - Custom LogisticRegression:
[[2457    0]
 [2543    0]]
              precision    recall  f1-score   support

    Positive       0.49      1.00      0.66      2457
    Negative       0.00      0.00      0.00      2543

    accuracy                           0.49      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.24      0.49      0.32      5000



C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

Trial 5 - Sklearn LogisticRegression:
[[   0 2512]
 [   0 2488]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2512
    Negative       0.50      1.00      0.66      2488

    accuracy                           0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.33      5000

Trial 5 - Custom LogisticRegression:
[[   0 2512]
 [   0 2488]]
              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00      2512
    Negative       0.50      1.00      0.66      2488

    accuracy                           0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.33      5000

Average Confusion Matrix - Sklearn LogisticRegression:
[[ 491.4 2017.6]
 [ 508.6 1982.4]]
Average Confusion Matrix - Custom LogisticRegression:
[[ 491.4 2017.6]
 [ 508.6 1982.4]]


C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\drew1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati